Run this cell to initialize population synthesis

In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from synthesize import *
from preprocess import *


config_folder = "../configs/Chicago"
n = 1000

person = process_pums_data(config_folder)
population_sample = synthesize_population(
    config_folder=config_folder, n_sample=n, min_age=18, max_age=65, random_state=2
)
ploc = puma_locations(config_folder)

Run this cell to initalize template generation

In [4]:
smg = SystemMessageGenerator(config_folder, "SystemMessage.j2", verbose_debug=False, shuffle=True, wrap=80)
year = 2019

output = []
attribute_descriptions = get_attribute_descriptions(person)
for i, individual in population_sample.reset_index().iterrows():
    individual_attributes = attribute_decoder_dict(individual.to_dict(), person)
    system_message = smg.write_system_message(**individual_attributes, **attribute_descriptions, ploc=ploc, YEAR=year)
    print(system_message)
    print('\n')

You live in Worth, Illinois. Your highest educational attainment is a Master's
degree in English Language And Literature. You are a Local government employee
in elementary and secondary schools. In the last year you earned $78,000. You
typically work 40 hours per week and have worked for 52 weeks in the past year.
You live alone. You were born in 1968 in Canada and became a U.S. citizen by
naturalization in 2006. You are a 51 year old woman of Polish ancestry that
lives in Merrionette Park, Illinois.


You have been married for 11 years. You live in Hometown, Illinois. You were
born in the U.S. in 1969 in Illinois. You are a 50 year old man of Mexican
ancestry that lives in Palos Heights, Illinois. Your highest educational
attainment is an Associate's degree. You are an employee of a for-profit
business in sporting and athletic goods, and doll, toy and game. In the last
year you earned $69,000. You typically work 40 hours per week and have worked
for 52 weeks in the past year.


Your h

In [5]:
individual_attributes

{'index': 'MISSING',
 'RT': 'Person Record',
 'SERIALNO': '2019HU0044350',
 'DIVISION': 'East North Central (Midwest region)',
 'SPORDER': '02',
 'PUMA': '03526',
 'REGION': 'Midwest',
 'ST': 'Illinois/IL',
 'ADJINC': '2019 factor (1.010145)',
 'PWGTP': '563',
 'AGEP': '45',
 'CIT': 'Born in the U.S.',
 'CITWP': 'MISSING',
 'COW': 'Local government employee (city, county, etc.)',
 'DDRS': 'No',
 'DEAR': 'No',
 'DEYE': 'No',
 'DOUT': 'No',
 'DPHY': 'No',
 'DRAT': 'MISSING',
 'DRATX': 'MISSING',
 'DREM': 'No',
 'ENG': 'MISSING',
 'FER': 'MISSING',
 'GCL': 'No',
 'GCM': 'MISSING',
 'GCR': 'MISSING',
 'HIMRKS': 'Categorically ineligible for subsidy',
 'HINS1': 'Yes',
 'HINS2': 'No',
 'HINS3': 'No',
 'HINS4': 'Yes',
 'HINS5': 'No',
 'HINS6': 'No',
 'HINS7': 'No',
 'INTP': '0',
 'JWMNP': '30',
 'JWRIP': '1',
 'JWTRNS': 'Car, truck, or van',
 'LANX': 'No, speaks only English',
 'MAR': 'Married',
 'MARHD': 'No',
 'MARHM': 'Yes',
 'MARHT': 'Two times',
 'MARHW': 'No',
 'MARHYP': '2019',
 'MIG':